In [2]:
%pip install langchain langchain_openai PyPDF2 faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 KB 981.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 6.7 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [16]:
from PyPDF2 import PdfReader
from langchain_openai import AzureOpenAIEmbeddings, AzureOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
pdfreader = PdfReader('../data/Phuket.pdf')
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

# Show first 1000 characters of the text.
raw_text[:1000]

'Wat Phra Yai or Wat Phra \nPhuttha Ming Mongkhon \nLocated on top Khao Nak Koet \nand commonly called ‘the Big \nBuddha’ this 45m tall statue \nis revered by both the people \nof Phuket and visitors alike. \nThe lofty site offers the best \n360° views of the island, with \nsweeping vistas of Phuket \nTown, Kata, and Karon beaches, \nChalong Bay and more.\nOld Phuket Town \nThe Old Phuket Town is located \nalong Thalang, Phang-nga, \nYaowarat and Krabi roads. The \nbeginning of the tin boom in the \n19th century led to construction \nof many fine mansions and shops \nthat are still well preserved. The \narchitectural style, typical of the \nregion, is described as Sino-\nPortuguese and has a strongly \nMediterranean character. \nThe Upside Down House\nLocated on Bypass Road \n(Chalermphrakiat Road), called \nBan Teelanka in Thai, the pink \nhouse stands upside down \nand presents a fun attraction \nfor families and kids. Open \ndaily from 10am to 6pm. www.\nupsidedownhouse-phuket.comWa

In [11]:
# Split the text into chunks of 1000 characters with 200 characters overlap.
text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
pdfTexts = text_splitter.split_text(raw_text)
len(pdfTexts)

13

In [13]:
EMBEDDINGS_DEPLOYMENT_NAME = "text-embedding-ada-002"
EMBEDDINGS_API_VERSION = "2023-08-01-preview"

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=EMBEDDINGS_DEPLOYMENT_NAME,
    openai_api_version=EMBEDDINGS_API_VERSION,
)

In [14]:
document_search = FAISS.from_texts(pdfTexts, embeddings)


In [21]:
DEPLOYMENT_NAME="gpt-35-turbo-instruct"
MODEL_NAME = "gpt-35-turbo-instruct"
API_VERSION="2023-12-01-preview"
chain = load_qa_chain(AzureOpenAI(azure_deployment=DEPLOYMENT_NAME, model_name=MODEL_NAME, openai_api_version=API_VERSION), chain_type="stuff")


In [24]:
inquiry = "What activities can do in Phuket?"
docs = document_search.similarity_search(inquiry)
chain.invoke({"input_documents": docs, "question":inquiry})

{'input_documents': [Document(page_content='ascetic displays, including fire-walking and ascending sharp-bladed \nladders. \nLaguna Phuket Triathlon \nHeld in December at Hat Bang Tao, the triathlon attracts athletes from \nall over the world.\nactivities\nScuba Diving\nExpeditions regularly leave Phuket \nfor dives around neighbouring \nislands, and further afield, \nmost particularly around the \nenduringly popular Phi Phi Islands \nin neighbouring Krabi province \nand the Similan Islands in Phang-\nnga province. Diving in Phuket’s \nwarm clear blue waters is best \nfrom November to May, when \nthe calm seas and rain free days \nmake Phuket’s diving a truly unique \nexperience. \nSea Kayaking or Canoeing \nThis form of ecotourism is offered, \nas popular one-day tours or more \nextended tours involving overnight \ncamping on island beaches.'),
  Document(page_content='experience. \nSea Kayaking or Canoeing \nThis form of ecotourism is offered, \nas popular one-day tours or more \next